In [17]:
import cv2
import os, glob
import numpy as np
from PIL import Image
import torch as th
import torchvision

# open the video file
# vid_name = './videos/Anakin_Trim_2.mp4'

def sort_by_frame(path_list):
    frame_anno = []
    for p in path_list:
        # frame_idx = os.path.splitext(p.split('_')[-1])[0][5:]   # 0-4 is "frame", so we used [5:] here
        frame_idx = os.path.splitext(p.split('/')[-1].split('_')[-1])[0][5:]   # 0-4 is "frame", so we used [5:] here
        frame_anno.append(int(frame_idx))
    sorted_idx = np.argsort(frame_anno)
    sorted_path_list = []
    for idx in sorted_idx:
      sorted_path_list.append(path_list[idx])
    return sorted_path_list

track_name = 'Anakin_2'
composite_path = f"/home/mint/guided-diffusion/experiment_scripts/dealignment_ffhq/videos/composite/"
assert os.path.exists(composite_path)

print("[#] Loading composite images ... ", end='')
c_fl = glob.glob(f'/{composite_path}/f*')
c_fl = sort_by_frame(c_fl)
c_frames = th.stack([th.tensor(np.array(Image.open(f))) for f in c_fl], dim=0)
print("Done")

print("[#] Loading original images ... ", end='')
original_path = f"/data/mint/DPM_Dataset/Videos/{track_name}/images/"
o_fl = [original_path + f.split('/')[-1] for f in c_fl]
o_frames = th.stack([th.tensor(np.array(Image.open(f))) for f in o_fl], dim=0)
print("Done")

# torchvision.io.write_video(video_array=c_frames, filename=f'./out.avi', fps=25, video_codec='ffv1')
    
all_frames_type = th.cat((o_frames, c_frames), dim=1)
print(all_frames_type.shape)
print(th.max(all_frames_type))
print(th.min(all_frames_type))
os.makedirs(f'./out/{track_name}/')
for i, f in enumerate(c_fl):
    torchvision.utils.save_image(tensor=(all_frames_type[i]/255.0).permute(2, 0, 1).float(), fp=f"./out/{track_name}/comp_{f.split('/')[-1]}")
# torchvision.io.write_video(video_array=all_frames_type, filename=f'./combined.avi', fps=25, video_codec='ffv1')

[#] Loading composite images ... Done
[#] Loading original images ... Done
torch.Size([105, 3268, 3840, 3])
tensor(255, dtype=torch.uint8)
tensor(0, dtype=torch.uint8)
